In [2]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np


In [6]:
## Acessando pelo pacote da BD

df=bd.read_sql("SELECT  * FROM `basedosdados-projetos.republica.indicadores_estadic_perfil_gestor` where tema='educacao'", billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)

Downloading: 100%|██████████| 27/27 [00:00<00:00, 88.92rows/s]


In [44]:
df.head()

,ano,sigla_uf,tema,caracterizacao_orgao_gestor,sexo,idade,cor_raca,cor_raca_autodeclarado,escolaridade_gestor,escolaridade_formacao,faixa_etária
0,2018,RO,educacao,Secretaria exclusiva,Feminino,53,Branca,1,Especialização,Pedagogia,53-80
1,2018,AC,educacao,Secretaria em conjunto com outras políticas se...,Masculino,52,Branca,1,Mestrado,Letras,53-80
2,2018,AM,educacao,Secretaria exclusiva,Masculino,73,Branca,0,Mestrado,Direito,53-80
3,2018,RR,educacao,Secretaria exclusiva,Feminino,40,Branca,0,Especialização,Pedagogia,0
4,2018,PA,educacao,Secretaria exclusiva,Feminino,54,Pardo,1,Doutorado,Pedagogia,53-80


In [25]:
## mudando para escolaridade do gestor pois gestores costumam ter escolaridade acima de ensino superior, o que não faria sentido dentro das faixas que usamos em outros contextos. 
df = df.rename(columns={"escolaridade":"escolaridade_gestor"})

In [45]:
## agrupar faixa etária 


df['faixa_etária']=np.where((df['idade']>=18) & (df['idade']<30),"Entre 18 e 29 anos",0)
df['faixa_etária']=np.where((df['faixa_etária'] !=0) & (df['idade']>=30) & (df['idade']<50),"Entre 30 e 49 anos",df['faixa_etária'])
df['faixa_etária']=np.where((df['faixa_etária'] !=0) & (df['idade']>=50) & (df['idade']<65),"Entre 50 e 64 anos",df['faixa_etária'])
df['faixa_etária']=np.where((df['faixa_etária'] !=0) & (df['idade']>=65),"Acima de 65 anos",df['faixa_etária'])


In [46]:
df['faixa_etária'].unique()

array(['Entre 50 e 64 anos', 'Acima de 65 anos', 'Entre 30 e 49 anos'],
      dtype=object)

In [48]:
df.columns

Index(['ano', 'sigla_uf', 'tema', 'caracterizacao_orgao_gestor', 'sexo',
       'idade', 'cor_raca', 'cor_raca_autodeclarado', 'escolaridade_gestor',
       'escolaridade_formacao', 'faixa_etária'],
      dtype='object')

In [49]:
interesse = ['ano', 'sigla_uf', 'tema', 'caracterizacao_orgao_gestor', 'sexo',
       'idade','faixa_etária', 'cor_raca', 'cor_raca_autodeclarado', 'escolaridade_gestor',
       'escolaridade_formacao']

In [52]:
df1 = df[interesse]

In [54]:
df1.head()

,ano,sigla_uf,tema,caracterizacao_orgao_gestor,sexo,idade,faixa_etária,cor_raca,cor_raca_autodeclarado,escolaridade_gestor,escolaridade_formacao
0,2018,RO,educacao,Secretaria exclusiva,Feminino,53,Entre 50 e 64 anos,Branca,1,Especialização,Pedagogia
1,2018,AC,educacao,Secretaria em conjunto com outras políticas se...,Masculino,52,Entre 50 e 64 anos,Branca,1,Mestrado,Letras
2,2018,AM,educacao,Secretaria exclusiva,Masculino,73,Acima de 65 anos,Branca,0,Mestrado,Direito
3,2018,RR,educacao,Secretaria exclusiva,Feminino,40,Entre 30 e 49 anos,Branca,0,Especialização,Pedagogia
4,2018,PA,educacao,Secretaria exclusiva,Feminino,54,Entre 50 e 64 anos,Pardo,1,Doutorado,Pedagogia


In [ ]:
### Subindo para o GBQ

In [55]:
dataset_ref = client.dataset('cargos_lideranca')

NameError: name 'client' is not defined